This script runs on pre-processed, measured chlorophyll data.
The script "process_chlorophyll_data.R" generates these processed data from the raw data files.

This script generates the depth integrated chlorophyll data for each station where a chl measurement was made
and creates the csv file '../processed_data/depth_integrated_chl_by_station.csv'

In [100]:
import numpy as np
import pandas as pd

In [101]:
preprocessed_data_dir = '../data_preprocessed/'

metadata_dir = '../data_station_meta/'

In [102]:
bottle_all = pd.read_csv('../processed_data/final_qc_bottle_file.csv')
bottle_poc_pon = pd.read_csv('../data_preprocessed/ToTS_bottle_POC.csv', skiprows=1)[['STNNBR', 'CASTNO', 'SAMPNO', 'BTLNBR', 'BTLNBR_FLAG_W', 'POC', 'POC_FLAG_W', 'PON', 'PON_FLAG_W']]

In [103]:
bottle_poc_pon.columns

Index(['STNNBR', 'CASTNO', 'SAMPNO', 'BTLNBR', 'BTLNBR_FLAG_W', 'POC',
       'POC_FLAG_W', 'PON', 'PON_FLAG_W'],
      dtype='object')

In [104]:
ctd = pd.merge(bottle_all, bottle_poc_pon, how='left')

In [105]:
ctd.columns

Index(['EXPOCODE', 'SECT_ID', 'STNNBR', 'CASTNO', 'SAMPNO', 'BTLNBR',
       'BTLNBR_FLAG_W', 'DATE', 'TIME', 'LATITUDE', 'LONGITUDE', 'DEPTH',
       'CTDPRS_DBAR_BOTTLE', 'CTDTMP_ITS-90', 'CTDTMP2_ITS-90', 'CTDSAL_PSU',
       'CTDSAL_FLAG_W', 'CTDSAL2_PSU', 'CTDSAL2_FLAG_W', 'CTDOXY_ML/L',
       'CTDOXY_FLAG_W', 'CTDOXY2_ML/L', 'CTDOXY2_FLAG_W', 'CTDFLUOR_MG/M^3',
       'CTDFLUOR_FLAG_W', 'CTDXMISS_%TRANS', 'CTDXMISS_FLAG_W',
       'PAR_UEIN/M^2/S', 'PAR_FLAG_W', 'PAR_MAST_UEIN/M^2/S', 'PAR_MAST_FLAG_',
       'CHLORA_UG/L', 'CHLORA_FLAG_W', 'CHLORA_STDEV_UG/L',
       'CHLORA_STDEV_FLAG_W', 'CHLORA_CV_%', 'CHLORA_CV_FLAG_W', 'PHAEO_UG/L',
       'PHAEO_FLAG_W', 'PHAEO_STDEV_UG/L', 'PHAEO_STDEV_FLAG_W', 'PHAEO_CV_%',
       'PHAEO_CV_FLAG_W', 'Calvin#', 'Calvin#_FLAG_W', 'PHSPHT_UMOL/L',
       'PHSPHT_FLAG_W', 'SILCAT_UMOL/L', 'SILCAT_FLAG_W', 'NITRAT_UMOL/L',
       'NITRAT_FLAG_W', 'NITRIT_UMOL/L', 'NITRIT_FLAG_W', 'NH4_UMOL/L',
       'NH4_FLAG_W', 'DIN_UMOL/L', 'DIN_Flag_W',

In [106]:
ctd.loc[ctd['STNNBR'] == 137, 'CHLORA_UG/L']

514    0.24
515    0.52
516    0.72
517    0.77
Name: CHLORA_UG/L, dtype: float64

In [107]:
bottle_all.columns

Index(['EXPOCODE', 'SECT_ID', 'STNNBR', 'CASTNO', 'SAMPNO', 'BTLNBR',
       'BTLNBR_FLAG_W', 'DATE', 'TIME', 'LATITUDE', 'LONGITUDE', 'DEPTH',
       'CTDPRS_DBAR_BOTTLE', 'CTDTMP_ITS-90', 'CTDTMP2_ITS-90', 'CTDSAL_PSU',
       'CTDSAL_FLAG_W', 'CTDSAL2_PSU', 'CTDSAL2_FLAG_W', 'CTDOXY_ML/L',
       'CTDOXY_FLAG_W', 'CTDOXY2_ML/L', 'CTDOXY2_FLAG_W', 'CTDFLUOR_MG/M^3',
       'CTDFLUOR_FLAG_W', 'CTDXMISS_%TRANS', 'CTDXMISS_FLAG_W',
       'PAR_UEIN/M^2/S', 'PAR_FLAG_W', 'PAR_MAST_UEIN/M^2/S', 'PAR_MAST_FLAG_',
       'CHLORA_UG/L', 'CHLORA_FLAG_W', 'CHLORA_STDEV_UG/L',
       'CHLORA_STDEV_FLAG_W', 'CHLORA_CV_%', 'CHLORA_CV_FLAG_W', 'PHAEO_UG/L',
       'PHAEO_FLAG_W', 'PHAEO_STDEV_UG/L', 'PHAEO_STDEV_FLAG_W', 'PHAEO_CV_%',
       'PHAEO_CV_FLAG_W', 'Calvin#', 'Calvin#_FLAG_W', 'PHSPHT_UMOL/L',
       'PHSPHT_FLAG_W', 'SILCAT_UMOL/L', 'SILCAT_FLAG_W', 'NITRAT_UMOL/L',
       'NITRAT_FLAG_W', 'NITRIT_UMOL/L', 'NITRIT_FLAG_W', 'NH4_UMOL/L',
       'NH4_FLAG_W', 'DIN_UMOL/L', 'DIN_Flag_W',

In [108]:
ctd = ctd.rename(columns={'SAMPNO': 'sampleNum',
                          'STNNBR': 'station',
                          'CTDPRS_DBAR_BOTTLE': 'depth_m',
                          'CHLORA_UG/L': 'chlMean',
                          'PHAEO_UG/L': 'phaeoMean',
                          'POC':'poc',
                          'PON':'pon',
                          'DEPTH': 'waterColDepth'})
                        #   'CTDPRS_DBAR_BOTTLE': 'depth_m',
                        #   'CHLORA_UG/L': 'chlMean',
                        #   'PHAEO_UG/L': 'phaeoMean',
                        #   'DEPTH': 'waterColDepth'})

# meaning just eliminates duplicate samples at the same station/depth - chl and phaeo are taken in triplicate so actual mean there
poc_mean = ctd[ctd['POC_FLAG_W']==2].groupby(['station', 'depth_m'])['poc'].mean().reset_index()
pon_mean = ctd[ctd['PON_FLAG_W']==2].groupby(['station', 'depth_m'])['pon'].mean().reset_index()
chl_mean = ctd[ctd['CHLORA_FLAG_W']==6].groupby(['station', 'depth_m'])['chlMean'].mean().reset_index()
phaeo_mean = ctd[ctd['PHAEO_FLAG_W']==6].groupby(['station', 'depth_m'])['phaeoMean'].mean().reset_index()
waterColDepth = ctd[['station', 'waterColDepth']].drop_duplicates().sort_values(by='station')

ctd_poc = pd.merge(poc_mean, waterColDepth, on='station', how='left')
ctd_pon = pd.merge(pon_mean, waterColDepth, on='station', how='left')
ctd_chl = pd.merge(chl_mean, waterColDepth, on='station', how='left')
ctd_merge_pocpon = pd.merge(poc_mean, pon_mean, on=['station', 'depth_m'], how='outer')
ctd_merge_pigments = pd.merge(chl_mean, phaeo_mean, on=['station', 'depth_m'], how='outer').merge(waterColDepth, how='left')

print(len(ctd), len(ctd_poc), len(ctd_pon), len(ctd_chl))

1055 440 440 1059


In [109]:
ctd_chl

,station,depth_m,chlMean,waterColDepth
0,1.0,2.275,0.84,26.0
1,1.0,5.172,0.73,26.0
2,1.0,10.188,0.84,26.0
3,1.0,15.315,1.41,26.0
4,1.0,24.820,1.48,26.0
...,...,...,...,...
1054,298.0,12.392,1.13,29.0
1055,298.0,26.523,1.22,29.0
1056,299.0,2.026,0.85,25.0
1057,299.0,6.862,0.75,25.0


note: as read in chla is in units ug/L

I am pretty sure POC/PON is in umol/L here - going to operate under this assumption - CORRECTION: I think C and N are in ug/L

In [110]:
# Useful Constants for conversion - IF POC/PON in umol/L, then molar mass conversion needed
# molMassN = 14.0067
# molMassC = 12.011

# If POC/PON in ug/L, then no molar mass conversion needed
molMassN = 1
molMassC = 1

we do conversions here in the depth integration loop to convert to mg/m^2

In [111]:
## Note: there is no CTD cast at the deployment station for T026 (station 174)
## Data from station 167 (same day, just hours earlier) will be used for T026

traps = pd.read_csv('../data_station_meta/trap_metadata.csv')
traps.loc[traps['Trap_Depth'] == 0, 'Trap_Depth'] = 1

traps_trimmed = traps[['Sample_Number', 'Deployment', 'Section_Name', 'Deploy_Station', 'Trap_Depth']].dropna()

# Replace the deployment station for trap 26 with station 167
traps_trimmed.loc[traps_trimmed['Deployment']=='T026', 'Deploy_Station'] = 167

# traps_trimmed

In [112]:
for trap in traps_trimmed['Sample_Number']: # one loop for each trap
    meta = traps_trimmed.loc[traps_trimmed['Sample_Number'] == trap] # select only data for that trap
    trap_depth = meta['Trap_Depth'].iloc[0] # get the trap depth
    deploy_station = meta['Deploy_Station'].iloc[0] # deployment station
    deployment = meta['Deployment'].iloc[0] # deployment
    section = meta['Section_Name'].iloc[0] # section
    print(meta)

  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
0         T001S       T001          CN7              15           1
  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
1         T001D       T001          CN7              15          20
  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
2         T002S       T002          HR1              16          15
  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
3         T002D       T002          HR1              16          30
  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
4         T003S       T003          HR6              21           3
  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
5         T003D       T003          HR6              21          29
  Sample_Number Deployment Section_Name  Deploy_Station  Trap_Depth
6         T004S       T004         HRE2              26          15
  Sample_Number Deployment Section_Name  Deploy_

In [113]:
## rename this
def compute_mass_in_rectangle(heightRectangle_m, concentration, molarMass):
    wcVolume = heightRectangle_m * 1000 # meters tall assuming 1 m^2 area = m^3. m^3 *1000 to convert to L
    return {
        'integrated_mg_m2': wcVolume * concentration * molarMass / 1000 # volume (L) * concentration (umol/L) * molarMass (g/mol) / conversion factor (1000 ug/mg)
    }


In [114]:
def depth_integrate_variable(ctd_dataframe,
                             variable,
                             traps_trimmed = traps_trimmed):
    if variable == 'poc':
        molMass = molMassC
        var = 'poc'
    elif variable == 'pon':
        molMass = molMassN
        var = 'pon'
    elif variable == 'chla':
        molMass = 1
        var = 'chlMean'
    else:
        raise ValueError("Please enter 'poc', 'pon', or 'chla' as variable")
    
    di_above_trap_list = []

    for trap in traps_trimmed['Sample_Number']: # one loop for each trap
        meta = traps_trimmed.loc[traps_trimmed['Sample_Number'] == trap] # select only data for that trap
        trap_depth = meta['Trap_Depth'].iloc[0] # get the trap depth
        # print(meta)
        deploy_station = meta['Deploy_Station'].iloc[0] # deployment station
        deployment = meta['Deployment'].iloc[0] # deployment
        section = meta['Section_Name'].iloc[0] # section

        df_all_depths = ctd_dataframe.loc[ctd_dataframe['station'] == deploy_station] # return all measurements from trap deployment station
        
        if df_all_depths.empty:
            print(f"{trap}: No CTD data at station {deploy_station}")
            continue

        min_meas_depth = np.min(df_all_depths['depth_m'])
        max_depth = max(min_meas_depth, trap_depth) # if trap is shallower than first measurement, first measurement is max_depth. otherwise trap depth is max depth
        df = df_all_depths[df_all_depths['depth_m'] <= max_depth+1]
    
        if df.empty:
            print(f"{trap}: No CTD data shallower than max depth") # This is super strange occurance, and should only happen with no data
            continue

        rectangle = []

        max_meas_depth = np.max(df['depth_m'])

        # === CASE 1: Measured at trap depth ===
        if np.isclose(max_meas_depth, trap_depth, atol=1):
        # if np.isclose(np.max(df['depth_m']), trap_depth, atol=1):
            print(trap, 'measurement at trap depth')
            for j in range(len(df)):
                if j == 0:
                    depth = df['depth_m'].iloc[j]
                    conc = df[var].iloc[j]

                else:
                    depth = df['depth_m'].iloc[j] - df['depth_m'].iloc[j-1]
                    conc = np.nanmean([df[var].iloc[j], df[var].iloc[j-1]])

                rectangle.append(pd.DataFrame([compute_mass_in_rectangle(depth, conc, molMass)]))

        # === CASE 2: Trap is deeper than deepest measurement ===
        elif max_meas_depth < trap_depth:
            print(trap, 'no measurement at trap depth')
            for j in range(len(df)):
                if j == 0:
                    depth = df['depth_m'].iloc[j]
                    conc = df[var].iloc[j]

                elif j == len(df) - 1:
                    depth = trap_depth - df['depth_m'].iloc[j]
                    conc = df[var].iloc[j]

                else:
                    depth = df['depth_m'].iloc[j] - df['depth_m'].iloc[j-1]
                    conc = np.nanmean([df[var].iloc[j], df[var].iloc[j-1]])

                rectangle.append(pd.DataFrame([compute_mass_in_rectangle(depth, conc, molMass)]))

        # === CASE 3: Trap shallower than first CTD measurement ===
        elif min_meas_depth > trap_depth:
            print(trap, 'Trap shallower than first measurement — likely ice-tethered')
            depth = trap_depth
            conc = df[var].iloc[0]
            rectangle.append(pd.DataFrame([compute_mass_in_rectangle(depth, conc, molMass)]))

        else:
            print(trap, 'Unrecognized case — skipping')
            continue

        rectangle_df = pd.concat(rectangle)
        sum_flux = rectangle_df.sum(skipna=False)  # returns NaN if *all* are NaN in a column

        di_above_trap_list.append(pd.DataFrame({
        'trap': [trap],
        'deployment': [deployment],
        'station': [deploy_station],
        'section': [section],
        'trap_depth': [trap_depth],
        f'di{var}_mg_m2': [sum_flux['integrated_mg_m2']]
    }))

    # === Final Output ===
    return(pd.concat(di_above_trap_list).sort_values(by='trap'))
    # print(diChl_above_trap_df)
    # diChl_above_trap_df.to_csv('../results/depth_integrated_chl_above_traps.csv', index=False)


In [115]:
diPOC_mg_m2 = depth_integrate_variable(ctd_dataframe = ctd_poc, variable='poc', traps_trimmed = traps_trimmed)
diPON_mg_m2 = depth_integrate_variable(ctd_dataframe = ctd_pon, variable='pon', traps_trimmed = traps_trimmed)

print('Running Chla')
diChla_mg_m2 = depth_integrate_variable(ctd_dataframe = ctd_chl, variable='chla', traps_trimmed = traps_trimmed)

T001S: No CTD data at station 15
T001D: No CTD data at station 15
T002S measurement at trap depth
T002D no measurement at trap depth
T003S measurement at trap depth
T003D no measurement at trap depth
T004S no measurement at trap depth
T004D no measurement at trap depth
T005S no measurement at trap depth
T005D no measurement at trap depth
T006S Trap shallower than first measurement — likely ice-tethered
T006D no measurement at trap depth
T007S Trap shallower than first measurement — likely ice-tethered
T007D no measurement at trap depth
T008S Trap shallower than first measurement — likely ice-tethered
T008D no measurement at trap depth
T009S no measurement at trap depth
T009D no measurement at trap depth
T010S no measurement at trap depth
T010D measurement at trap depth
T011S: No CTD data at station 61
T011D: No CTD data at station 61
T012S Trap shallower than first measurement — likely ice-tethered
T012D no measurement at trap depth
T013S Trap shallower than first measurement — likely 

In [116]:
di_biomass_above_traps = pd.merge(diPOC_mg_m2, diPON_mg_m2, how = 'outer').merge(diChla_mg_m2, how='outer')

## Saved later with full water column data as well
# di_biomass_above_traps.to_csv('../processed_data/depth_integrated_biomass_above_traps.csv', index = False)

add some code for a full water column integration

In [117]:
def depth_integrate_full_water_column(ctd_dataframe, variable):
    # Select molar mass and CTD variable
    if variable == 'poc':
        mol_mass = molMassC
        ctd_var = 'poc'
    elif variable == 'pon':
        mol_mass = molMassN
        ctd_var = 'pon'
    elif variable == 'chla':
        mol_mass = 1
        ctd_var = 'chlMean'
    else:
        raise ValueError("Please enter 'poc', 'pon', or 'chla' as variable")

    results = []

    for station, df in ctd_dataframe.groupby('station'):
        df = df.sort_values('depth_m').reset_index(drop=True)
        water_col_depth = df['waterColDepth'].iloc[0]  # assume consistent within station

        rectangles = []

        for i in range(len(df)):
            if i == 0:
                # surface layer — from surface to first measurement
                depth = df['depth_m'].iloc[i]
                conc = df[ctd_var].iloc[i]
            else:
                depth = df['depth_m'].iloc[i] - df['depth_m'].iloc[i - 1]
                conc = np.nanmean([df[ctd_var].iloc[i], df[ctd_var].iloc[i - 1]])

            rectangles.append(pd.DataFrame([compute_mass_in_rectangle(depth, conc, mol_mass)]))

        # Add final rectangle from deepest measurement to seafloor
        deepest_meas_depth = df['depth_m'].iloc[-1]
        if water_col_depth > deepest_meas_depth:
            bottom_depth = water_col_depth - deepest_meas_depth
            bottom_conc = df[ctd_var].iloc[-1]
            rectangles.append(pd.DataFrame([compute_mass_in_rectangle(bottom_depth, bottom_conc, mol_mass)]))

        if rectangles:
            integrated = pd.concat(rectangles).sum(skipna=False)
            results.append(pd.DataFrame({
                'station': [station],
                f'di{ctd_var}_full_waterCol_mg_m2': [integrated['integrated_mg_m2']]
            }))

    return pd.concat(results).sort_values(by='station')


In [118]:
diPOC_mg_m2_fullWC = depth_integrate_full_water_column(ctd_dataframe = ctd_poc, variable='poc')
diPON_mg_m2_fullWC = depth_integrate_full_water_column(ctd_dataframe = ctd_pon, variable='pon')
diChla_mg_m2_fullWC = depth_integrate_full_water_column(ctd_dataframe = ctd_chl, variable='chla')

In [119]:
di_biomass = pd.merge(diPOC_mg_m2_fullWC, diPON_mg_m2_fullWC, how = 'outer').merge(diChla_mg_m2_fullWC, how='outer')
di_biomass

,station,dipoc_full_waterCol_mg_m2,dipon_full_waterCol_mg_m2,dichlMean_full_waterCol_mg_m2
0,16.0,2254.989047,290.695172,671.927875
1,19.0,1126.877655,168.064990,360.100830
2,21.0,813.418123,91.607116,57.466420
3,22.0,394.263217,51.666191,79.496340
4,23.0,797.869619,72.323826,106.630195
...,...,...,...,...
228,295.0,NaN,NaN,38.124740
229,296.0,NaN,NaN,26.029515
230,297.0,NaN,NaN,39.002885
231,298.0,NaN,NaN,28.319900


In [120]:
di_biomass.to_csv('../processed_data/depth_integrated_biomass_full_wc_all_stations.csv', index = False)

In [121]:
di_biomass_above_traps = pd.merge(diPOC_mg_m2, diPON_mg_m2, how = 'outer').merge(diChla_mg_m2, how='outer')
di_biomass_above_traps = pd.merge(di_biomass_above_traps, di_biomass, how='left')
# di_biomass_above_traps.to_csv('../results/depth_integrated_biomass_above_traps.csv', index = False)
di_biomass_above_traps.to_csv('../processed_data/depth_integrated_biomass_above_traps.csv', index = False)

In [122]:
di_biomass_above_traps

,trap,deployment,station,section,trap_depth,dipoc_mg_m2,dipon_mg_m2,dichlMean_mg_m2,dipoc_full_waterCol_mg_m2,dipon_full_waterCol_mg_m2,dichlMean_full_waterCol_mg_m2
0,T002D,T002,16,HR1,30,1193.859831,131.510134,313.230335,2254.989047,290.695172,671.927875
1,T002S,T002,16,HR1,15,978.835437,96.604933,229.906745,2254.989047,290.695172,671.927875
2,T003D,T003,21,HR6,29,332.947815,40.897044,40.499200,813.418123,91.607116,57.466420
3,T003S,T003,21,HR6,3,29.358232,4.508769,6.063720,813.418123,91.607116,57.466420
4,T004D,T004,26,HRE2,25,229.124388,27.248944,17.179840,920.884407,118.797804,61.338665
5,T004S,T004,26,HRE2,15,96.256171,11.634326,3.252390,920.884407,118.797804,61.338665
6,T005D,T005,29,HRE5,30,688.090066,62.042383,157.490640,1614.383670,169.410432,390.935275
7,T005S,T005,29,HRE5,15,363.206981,34.579255,93.438260,1614.383670,169.410432,390.935275
8,T006D,T006,32,HRE8,35,290.084667,45.054069,87.901200,539.812997,79.616725,135.103555
9,T006S,T006,32,HRE8,1,19.356589,3.215947,6.780000,539.812997,79.616725,135.103555
